<a href="https://colab.research.google.com/github/TarteTatin1212/face_recognition_script2/blob/main/face_recognition_script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p "/content/drive/MyDrive/Horita_Semi/Senior_Project/models/"


In [ ]:
import requests

# YuNet モデルのURL
yunet_model_url = 'https://raw.githubusercontent.com/ShiqiYu/libfacedetection.train/master/onnx/yunet_s_640_640.onnx'

# モデルをダウンロードして保存するファイルパス
model_save_path = '/content/drive/MyDrive//Horita_Semi/Senior_Project/models/yunet.onnx'

# モデルのダウンロード
response = requests.get(yunet_model_url)
if response.status_code == 200:
    with open(model_save_path, 'wb') as file:
        file.write(response.content)
    print("YuNet モデルがダウンロードされました。")
else:
    print("YuNet モデルのダウンロードに失敗しました。ステータスコード:", response.status_code)


YuNet モデルがダウンロードされました。


In [ ]:
import cv2

def load_yunet_model(model_path):
    try:
        model = cv2.dnn.readNetFromONNX(model_path)
        return model
    except Exception as e:
        print(f"モデルのロード中にエラーが発生しました: {e}")
        return None

def get_model_layer_names(model):
    try:
        layer_names = model.getLayerNames()
        return layer_names
    except Exception as e:
        print(f"レイヤー名の取得中にエラーが発生しました: {e}")
        return []

# YuNet モデルのパスを指定
model_path = '/content/drive/MyDrive//Horita_Semi/Senior_Project/models/yunet.onnx'

# YuNet モデルをロード
model = load_yunet_model(model_path)
if model is not None:
    # モデルのレイヤー名を表示
    layer_names = get_model_layer_names(model)
    if layer_names:
        for i, layer_name in enumerate(layer_names):
            print(i, layer_name)
    else:
        print("レイヤー名は取得できませんでした。")
else:
    print("モデルはロードできませんでした。")


0 onnx_node!Conv_0
1 onnx_node!Relu_1
2 onnx_node!Conv_2
3 onnx_node!Conv_3
4 onnx_node!Relu_4
5 onnx_node!MaxPool_5
6 onnx_node!Conv_6
7 onnx_node!Conv_7
8 onnx_node!Relu_8
9 onnx_node!Conv_9
10 onnx_node!Conv_10
11 onnx_node!Relu_11
12 onnx_node!Conv_12
13 onnx_node!Conv_13
14 onnx_node!Relu_14
15 onnx_node!Conv_15
16 onnx_node!Conv_16
17 onnx_node!Relu_17
18 onnx_node!MaxPool_18
19 onnx_node!Conv_19
20 onnx_node!Conv_20
21 onnx_node!Relu_21
22 onnx_node!Conv_22
23 onnx_node!Conv_23
24 onnx_node!Relu_24
25 onnx_node!MaxPool_25
26 onnx_node!Conv_26
27 onnx_node!Conv_27
28 onnx_node!Relu_28
29 onnx_node!Conv_29
30 onnx_node!Conv_30
31 onnx_node!Relu_31
32 onnx_node!MaxPool_32
33 onnx_node!Conv_33
34 onnx_node!Conv_34
35 onnx_node!Relu_35
36 onnx_node!Conv_36
37 onnx_node!Conv_37
38 onnx_node!Relu_38
39 onnx_node!Conv_39
40 onnx_node!Conv_40
41 onnx_node!Relu_41
42 onnx_node!Resize_43
43 onnx_node!Add_44
44 onnx_node!Conv_45
45 onnx_node!Conv_46
46 onnx_node!Relu_47
47 onnx_node!Resize_

In [ ]:
!pip install opencv-python-headless

In [ ]:
import os
import cv2
import shutil
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def perform_face_detection_yunet(images_dir, base_dir):
    # YuNetモデルの初期化
    try:
        model = cv2.dnn.readNetFromONNX('/content/drive/MyDrive/Horita_Semi/Senior_Project/models/yunet.onnx')
    except Exception as e:
        print(f"モデルの読み込み中にエラーが発生しました: {e}")
        return

    # ディレクトリを作成
    new_folder = os.path.join(base_dir, 'YuNetFaceDetectionResults')
    detected_faces_dir = os.path.join(new_folder, 'DetectedFaces')
    undetected_faces_dir = os.path.join(new_folder, 'UndetectedFaces')
    os.makedirs(detected_faces_dir, exist_ok=True)
    os.makedirs(undetected_faces_dir, exist_ok=True)

    # images_dirにある各ファイルに対して処理を行う
    for filename in os.listdir(images_dir):
        if filename.endswith(".jpg"):
            filepath = os.path.join(images_dir, filename)

            image = cv2.imread(filepath)
            if image is None:
                print(f"画像が読み込めません: {filepath}")
                continue

            h, w = image.shape[:2]

            # 画像の前処理
            blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(320, 320), mean=(104.0, 177.0, 123.0))
            model.setInput(blob)

            # 顔検出の実行
            confidences, boxes = model.forward(["conf", "loc"])

            if len(boxes) > 0:
                for i in range(boxes.shape[0]):
                    box = boxes[i, :]
                    x1, y1, x2, y2 = box[0] * w, box[1] * h, box[2] * w, box[3] * h
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                save_path = os.path.join(detected_faces_dir, filename)
                cv2.imwrite(save_path, image)
            else:
                shutil.copy(filepath, os.path.join(undetected_faces_dir, filename))


In [ ]:
if __name__ == "__main__":
    # ディレクトリを変更
    images_directory = '/content/drive/MyDrive/Horita_Semi/Senior_Project/thumbnails'
    base_directory = '/content/drive/MyDrive/Horita_Semi/Senior_Project'

    # perform_face_detection_yunet関数を呼び出す
    perform_face_detection_yunet(images_directory, base_directory)


error: ignored